In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from keras.layers import LSTM,Dense,Input,Embedding, Bidirectionalal, Dropout
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras import metrics

In [9]:
input_lines = []
target_lines = []
for line in open('robert_frost.txt'):
    line = (line.rstrip())
    
    if not line:
        continue
    
    input_line = '<sos> '+ line
    target_line = line + ' <eos>'
    
    input_lines.append(input_line)
    target_lines.append(target_line)
    

In [10]:
input_line = []
for line in open('robert_frost.txt'):
    input_line.append(line.rstrip())

In [14]:
all_lines = input_lines + target_lines  # for tokenizing

In [15]:
tokenizer = Tokenizer(filters = '')

In [16]:
tokenizer.fit_on_texts(all_lines)
input_lines = tokenizer.texts_to_sequences(input_lines)
target_lines = tokenizer.texts_to_sequences(target_lines)

In [17]:
(input_linesa) = np.array(input_lines)
target_linesa = np.array(target_lines)

In [22]:
maxlength = max(len(s) for s in input_linesa)

In [24]:
input_lines = pad_sequences(input_lines,maxlength, padding='post',
    truncating='post')
target_lines = pad_sequences(target_lines,maxlength, padding='post',
    truncating='post')

In [29]:
cd C:\\Users\\senthilkumar\\Desktop\\ML

C:\Users\senthilkumar\Desktop\ML


In [30]:
#embedding vecs
word2vec = {}
with open('glove.6B.100d.txt',encoding ="utf8") as f:
    for line in f:
        values = line.split()
        vec = np.asarray(values[1:],dtype = 'float32')
        word = values[0]
        word2vec[word] = vec

In [31]:
len(word2vec)

400000

In [32]:
vocab_size= len(tokenizer.word_index)+1

In [35]:
target_lines_onehot = to_categorical(target_lines,num_classes=vocab_size )

In [36]:
target_lines_onehot.shape

(1436, 12, 3057)

In [37]:
(target_lines_onehot[1,:,].shape) 

(12, 3057)

In [40]:
pwd

'C:\\Users\\senthilkumar\\Desktop\\ML'

In [43]:
#MOdel

In [44]:
#making the embedding matrix to be put into embedding layer
embedding_matrix = np.zeros((vocab_size,100))
for i,word in tokenizer.index_word.items():
    vec = word2vec.get(word)
    if vec  is not None:
        embedding_matrix[i,:] = vec

In [48]:
LATENT_DIM = 25

In [49]:
embedding = Embedding(vocab_size,100,weights = [embedding_matrix],trainable = True)

In [50]:
input_ = Input(shape = (maxlength,))
initial_h = Input(shape = (LATENT_DIM,))
initial_c = Input(shape  = (LATENT_DIM,))
x = embedding(input_)
lstm_ = LSTM(LATENT_DIM,return_sequences=True,return_state=True)
x,_,_ = lstm_(x,initial_state = [initial_h,initial_c])
dense_ = Dense(vocab_size,activation = 'softmax')
output = dense_(x)

Instructions for updating:
Colocations handled automatically by placer.


In [51]:
model = Model([input_,initial_h,initial_c],output)
model.compile(loss = 'categorical_crossentropy',optimizer = Adam(lr = 0.01),metrics = ['accuracy'])

In [55]:
z = np.zeros((len(input_lines),LATENT_DIM))

In [84]:
z = np.zeros((len(input_lines),LATENT_DIM))
r= model.fit([input_lines,z,z],target_lines_onehot,batch_size = 128,epochs = 150,validation_split=0.1)

Train on 1292 samples, validate on 144 samples
Epoch 1/150
1292/1292 [==============================] - 9s 7ms/step - loss: 4.3412 - acc: 0.3183 - val_loss: 5.7282 - val_acc: 0.2656
Epoch 2/150
1292/1292 [==============================] - 7s 6ms/step - loss: 4.3251 - acc: 0.3368 - val_loss: 5.7271 - val_acc: 0.2587
Epoch 3/150
1292/1292 [==============================] - 7s 6ms/step - loss: 4.3104 - acc: 0.3199 - val_loss: 5.7383 - val_acc: 0.2656
Epoch 4/150
1292/1292 [==============================] - 9s 7ms/step - loss: 4.2933 - acc: 0.3409 - val_loss: 5.7656 - val_acc: 0.2685
Epoch 5/150
1292/1292 [==============================] - 8s 6ms/step - loss: 4.2824 - acc: 0.3477 - val_loss: 5.7785 - val_acc: 0.2703
Epoch 6/150
1292/1292 [==============================] - 6s 5ms/step - loss: 4.2667 - acc: 0.3595 - val_loss: 5.7937 - val_acc: 0.2662
Epoch 7/150
1292/1292 [==============================] - 8s 6ms/step - loss: 4.2591 - acc: 0.3617 - val_loss: 5.7885 - val_acc: 0.2755
Epoch 8/

Epoch 61/150
1292/1292 [==============================] - 10s 7ms/step - loss: 2.7978 - acc: 0.4891 - val_loss: 5.7116 - val_acc: 0.3472
Epoch 62/150
1292/1292 [==============================] - 10s 8ms/step - loss: 2.7762 - acc: 0.4905 - val_loss: 5.7299 - val_acc: 0.3443
Epoch 63/150
1292/1292 [==============================] - 10s 7ms/step - loss: 2.7536 - acc: 0.4919 - val_loss: 5.7404 - val_acc: 0.3438
Epoch 64/150
1292/1292 [==============================] - 8s 6ms/step - loss: 2.7301 - acc: 0.4951 - val_loss: 5.7313 - val_acc: 0.3461
Epoch 65/150
1292/1292 [==============================] - 8s 6ms/step - loss: 2.7115 - acc: 0.4983 - val_loss: 5.7772 - val_acc: 0.3426
Epoch 66/150
1292/1292 [==============================] - 9s 7ms/step - loss: 2.6883 - acc: 0.4997 - val_loss: 5.7413 - val_acc: 0.3461
Epoch 67/150
1292/1292 [==============================] - 9s 7ms/step - loss: 2.6696 - acc: 0.5052 - val_loss: 5.7992 - val_acc: 0.3461
Epoch 68/150
1292/1292 [=====================

1292/1292 [==============================] - 7s 6ms/step - loss: 2.0032 - acc: 0.5886 - val_loss: 6.2698 - val_acc: 0.3466
Epoch 122/150
1292/1292 [==============================] - 6s 5ms/step - loss: 1.9978 - acc: 0.5891 - val_loss: 6.3172 - val_acc: 0.3438
Epoch 123/150
1292/1292 [==============================] - 6s 5ms/step - loss: 1.9898 - acc: 0.5918 - val_loss: 6.3017 - val_acc: 0.3438
Epoch 124/150
1292/1292 [==============================] - 7s 5ms/step - loss: 1.9809 - acc: 0.5949 - val_loss: 6.3136 - val_acc: 0.3455
Epoch 125/150
1292/1292 [==============================] - 7s 5ms/step - loss: 1.9824 - acc: 0.5942 - val_loss: 6.3353 - val_acc: 0.3443
Epoch 126/150
1292/1292 [==============================] - 7s 5ms/step - loss: 1.9759 - acc: 0.5929 - val_loss: 6.3019 - val_acc: 0.3484
Epoch 127/150
1292/1292 [==============================] - 8s 6ms/step - loss: 1.9715 - acc: 0.5938 - val_loss: 6.3127 - val_acc: 0.3466
Epoch 128/150
1292/1292 [==============================

In [85]:
    input_generate = Input(shape  = (1,))
    x = embedding(input_generate)
    x,h,c = lstm_(x,initial_state = [initial_h,initial_c])
    output_ = dense_(x)
    sampling_model = Model([input_generate,initial_h,initial_c],[output_,h,c])
 

In [86]:
idx_to_word = tokenizer.index_word
word_to_idx = tokenizer.word_index

In [101]:
generate_sentences()

'he has one, had sure in snow and time'

In [92]:
def generate_sentences():
    
    inp = np.array([[ word_to_idx['<sos>'] ]])
    h = np.zeros((1,LATENT_DIM))
    c = np.zeros((1,LATENT_DIM))
    eos = word_to_idx['<eos>']
    output_sentence = []
    for _ in range(maxlength):
        if ((inp[0,0] == eos) and(len(output_sentence)<=3)):
            inp[0,0] = previous_word
        o,h,c = sampling_model.predict([inp,h,c])
        probs = o[0,0]
        #if np.argmax(probs) == 0:
            #print('lol')
        probs[0] = 0
        probs /= sum(probs)
        word_index= np.random.choice(len(probs),p = probs)
        if len(output_sentence)>3:
            if eos == word_index:
                break
        if(eos == word_index):
            continue
        #word = idx_to_word.get(word_index)
        output_sentence.append(idx_to_word.get(word_index, '<WTF %s>' % word_index))
        previous_word = inp[0,0]
        inp[0,0] = word_index
    return ' '.join(output_sentence)
    

In [103]:
def generate_parawise(paras):
    for _ in range(6):
        print('\n')
        for _ in range(paras):
            sent = (generate_sentences())
            print(sent)
            
        

In [104]:
generate_parawise(4) ### This is the final generated poem



it sailing up one of fountain a certain coolness came hudson's double
and technical.- and try
with having eased her sills they're with poems.
of one that is our wildest ears suggest it? of spray;


once one street,
it's too, son. no more
one that surer struck en masse
'i'm aftermath,


at kiting font bed
two roads diverged in on may, in the meeting say go think
and now left's do what will we know in earnest.
they weren't;


'i want what the ominous wood, if and stars,
half closes the buttons hillside your horse made him do
'you have ideas yet empty.
if a (now make me to make you _can_ one then going


makes on position they leave such a guide.'
like lightning or one, who wrote the birds
'the height of heart they came with being are keeping the feet
i thought of me a ones though,


a shattered statement of relationship,
a book of was roar
and frozen horse cellar hole,
'i must be mad.'
